In [1]:
import tensorflow as tf
import numpy as np
import os, sys
from tensorflow.python.training import moving_averages
import numpy, time, random
from pathlib import Path

import pygame, copy


In [2]:
UPDATE_OPS_COLLECTION = "_update_ops_"
num_channel = 4 # this should be "Even" number

# create variable
def create_variable(name, shape, initializer,
    dtype=tf.float32, trainable=True):
    return tf.get_variable(name, shape=shape, dtype=dtype,
            initializer=initializer, trainable=trainable)

# batchnorm layer
def bacthnorm(inputs, scope, epsilon=1e-05, momentum=0.99, is_training=True):
    inputs_shape = inputs.get_shape().as_list()
    params_shape = inputs_shape[-1:]
    axis = list(range(len(inputs_shape) - 1))

    with tf.variable_scope(scope):
        beta = create_variable("beta", params_shape,
                               initializer=tf.zeros_initializer())
        gamma = create_variable("gamma", params_shape,
                                initializer=tf.ones_initializer())
        # for inference
        moving_mean = create_variable("moving_mean", params_shape,
                            initializer=tf.zeros_initializer(), trainable=False)
        moving_variance = create_variable("moving_variance", params_shape,
                            initializer=tf.ones_initializer(), trainable=False)
    if is_training:
        mean, variance = tf.nn.moments(inputs, axes=axis)
        update_move_mean = moving_averages.assign_moving_average(moving_mean,
                                                mean, decay=momentum)
        update_move_variance = moving_averages.assign_moving_average(moving_variance,
                                                variance, decay=momentum)
        tf.add_to_collection(UPDATE_OPS_COLLECTION, update_move_mean)
        tf.add_to_collection(UPDATE_OPS_COLLECTION, update_move_variance)
    else:
        mean, variance = moving_mean, moving_variance
    return tf.nn.batch_normalization(inputs, mean, variance, beta, gamma, epsilon)

# depthwise conv2d layer
def depthwise_conv2d(inputs, scope, filter_size=3, channel_multiplier=1, strides=1):
    inputs_shape = inputs.get_shape().as_list()
    in_channels = inputs_shape[-1]
    with tf.variable_scope(scope):
        filter = create_variable("filter", shape=[filter_size, filter_size,
                                                  in_channels, channel_multiplier],
                       initializer=tf.truncated_normal_initializer(stddev=0.01))

    return tf.nn.depthwise_conv2d(inputs, filter, strides=[1, strides, strides, 1],
                                padding="SAME", rate=[1, 1])

# conv2d layer
def conv2d(inputs, scope, num_filters, filter_size=1, strides=1):
    inputs_shape = inputs.get_shape().as_list()
    in_channels = inputs_shape[-1]
    with tf.variable_scope(scope):
        filter = create_variable("filter", shape=[filter_size, filter_size,
                                                  in_channels, num_filters],
                        initializer=tf.truncated_normal_initializer(stddev=0.01))
    return tf.nn.conv2d(inputs, filter, strides=[1, strides, strides, 1],
                        padding="SAME")

# avg pool layer
def avg_pool(inputs, pool_size, scope):
    with tf.variable_scope(scope):
        return tf.nn.avg_pool(inputs, [1, pool_size, pool_size, 1],
                strides=[1, pool_size, pool_size, 1], padding="VALID")

# fully connected layer
def fc(inputs, n_out, scope, use_bias=True):
    inputs_shape = inputs.get_shape().as_list()
    n_in = inputs_shape[-1]
    with tf.variable_scope(scope):
        weight = create_variable("weight", shape=[n_in, n_out],
                    initializer=tf.random_normal_initializer(stddev=0.01))
        if use_bias:
            bias = create_variable("bias", shape=[n_out,],
                                   initializer=tf.zeros_initializer())
            return tf.nn.xw_plus_b(inputs, weight, bias)
        return tf.matmul(inputs, weight)

# main mobile net model
def model(inputs, num_classes=5, is_training_=True,
             width_multiplier=1, scope="MobileNet"):
    # 8 * 512 * 1
    inputs = tf.reshape(inputs, [-1,20,20,5])
    
    
    net = conv2d(inputs, "conv_1", round(32 * width_multiplier), filter_size=3,
                 strides=1)  # -> [N, 20, 20, 32]
    net = tf.nn.relu(bacthnorm(net, "conv_1/bn", is_training=is_training_))
    net = _depthwise_separable_conv2d(net, 64, width_multiplier,
                        "ds_conv_2") # -> [N, 20, 20, 64]
    net = _depthwise_separable_conv2d(net, 128, width_multiplier,
                        "ds_conv_3", downsample=True) # -> [N, 10, 10, 128]
    net = _depthwise_separable_conv2d(net, 128, width_multiplier,
                        "ds_conv_4") # -> [N, 10, 10, 128]
    net = _depthwise_separable_conv2d(net, 256, width_multiplier,
                        "ds_conv_5", downsample=False) # -> [N, 10, 10, 256]
    net = _depthwise_separable_conv2d(net, 256, width_multiplier,
                        "ds_conv_6") # -> [N, 10, 10, 256]
    net = _depthwise_separable_conv2d(net, 512, width_multiplier,
                        "ds_conv_7", downsample=True) # -> [N, 5, 5, 512]
    net = _depthwise_separable_conv2d(net, 512, width_multiplier,
                        "ds_conv_8") # -> [N, 5, 5, 512]
    net = _depthwise_separable_conv2d(net, 512, width_multiplier,
                        "ds_conv_9")  # -> [N, 5, 5, 512]
    net = _depthwise_separable_conv2d(net, 512, width_multiplier,
                        "ds_conv_10")  # -> [N, 5, 5, 512]
    net = _depthwise_separable_conv2d(net, 512, width_multiplier,
                        "ds_conv_11")  # -> [N, 5, 5, 512]
    net = _depthwise_separable_conv2d(net, 512, width_multiplier,
                        "ds_conv_12")  # -> [N, 5, 5, 512]
    net = _depthwise_separable_conv2d(net, 1024, width_multiplier,
                        "ds_conv_13", downsample=False) # -> [N, 5, 5, 1024]
    net = _depthwise_separable_conv2d(net, 1024, width_multiplier,
                        "ds_conv_14") # -> [N, 5, 5, 1024]
    net = avg_pool(net, 5, "avg_pool_15")
    net = tf.squeeze(net, [1, 2], name="SpatialSqueeze")
    logits = fc(net, num_classes, "fc_16")
    
    return logits

def _depthwise_separable_conv2d( inputs, num_filters, width_multiplier,
                                scope, downsample=False, is_training_ = True):
    """depthwise separable convolution 2D function"""
    num_filters = round(num_filters * width_multiplier)
    strides = 2 if downsample else 1

    with tf.variable_scope(scope):
        # depthwise conv2d
        dw_conv = depthwise_conv2d(inputs, "depthwise_conv", strides=strides)
        # batchnorm
        bn = bacthnorm(dw_conv, "dw_bn", is_training=is_training_)
        # relu
        relu = tf.nn.relu(bn)
        # pointwise conv2d (1x1)
        pw_conv = conv2d(relu, "pointwise_conv", num_filters)
        # bn
        bn = bacthnorm(pw_conv, "pw_bn", is_training=is_training_)
        return tf.nn.relu(bn)

In [3]:
def pygameInitializer():
    
    SCREEN_WIDTH = 640
    SCREEN_HEIGHT = 480

    white = (255, 255, 255)
    red = (255, 0, 0)
    blue = (0, 0, 255)
    green = (0, 255, 0)
    black = (0, 0, 0)
    colorList = [red, (150, 0, 0), (100, 30, 30), white]

    pygame.init()
    pygame.display.set_caption("Simple PyGame Example")
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

    # for polygon. the order is 1, 3, 4, 5, 2, counterclockwise
    Gamma = 0.2
    lineLengthPolygon = int(SCREEN_HEIGHT * Gamma)  # actually, it's half of lineLength of polygon.
    coordinatePolygonA = (int(0.5 * SCREEN_WIDTH), int((0.5 - 1.53885 * Gamma) * SCREEN_HEIGHT))
    coordinatePolygonE = (
        int(coordinatePolygonA[0] - 1.618 * lineLengthPolygon), int(coordinatePolygonA[1] + 1.1755 * lineLengthPolygon))
    coordinatePolygonU = (
        int(coordinatePolygonA[0] + 1.618 * lineLengthPolygon), int(coordinatePolygonA[1] + 1.1755 * lineLengthPolygon))
    coordinatePolygonI = (
        int(coordinatePolygonA[0] - lineLengthPolygon), int(coordinatePolygonA[1] + 3.0777 * lineLengthPolygon))
    coordinatePolygonO = (
        int(coordinatePolygonA[0] + lineLengthPolygon), int(coordinatePolygonA[1] + 3.0777 * lineLengthPolygon))

    # initial (center) position of polygon
    pos_x = int(SCREEN_WIDTH / 2)
    pos_y = int(coordinatePolygonI[1] - 1.37638192047 * lineLengthPolygon)
    # center position of window
    center_x = int(SCREEN_WIDTH / 2)
    center_y = int(coordinatePolygonI[1] - 1.37638192047 * lineLengthPolygon)

    # the lines for degree
    listsForDotsOfDegree = []
    numberOfDegree = 4
    forCountOfDegree = copy.copy(numberOfDegree)
    while forCountOfDegree != 0:
        listForDots = []
        firstDot = (coordinatePolygonA[0],
                    int((pos_y - coordinatePolygonA[1]) / (numberOfDegree + 1)) * forCountOfDegree + coordinatePolygonA[1])
        listForDots.append(firstDot)
        secondDot = (int((pos_x - coordinatePolygonU[0]) / (numberOfDegree + 1)) * forCountOfDegree + coordinatePolygonU[0],
                     int((pos_y - coordinatePolygonU[1]) / (numberOfDegree + 1)) * forCountOfDegree + coordinatePolygonU[1])
        listForDots.append(secondDot)
        thirdDot = (int((pos_x - coordinatePolygonE[0]) / (numberOfDegree + 1)) * forCountOfDegree + coordinatePolygonE[0],
                    int((pos_y - coordinatePolygonE[1]) / (numberOfDegree + 1)) * forCountOfDegree + coordinatePolygonE[1])
        listForDots.append(thirdDot)
        fourthDot = (
            int((pos_x - coordinatePolygonI[0]) / (numberOfDegree + 1)) * (forCountOfDegree) + coordinatePolygonI[0],
            int((coordinatePolygonI[1] - pos_y) / (numberOfDegree + 1)) * (numberOfDegree - forCountOfDegree + 1) + pos_y)
        listForDots.append(fourthDot)
        fifthDot = (
            int((coordinatePolygonO[0] - pos_x) / (numberOfDegree + 1)) * (numberOfDegree - forCountOfDegree + 1) + pos_x,
            int((coordinatePolygonO[1] - pos_y) / (numberOfDegree + 1)) * (numberOfDegree - forCountOfDegree + 1) + pos_y)
        listForDots.append(fifthDot)
        listsForDotsOfDegree.append(listForDots)
        forCountOfDegree -= 1

    # font
    font = 72
    fontObj1 = pygame.font.SysFont("comicsansms", font)
    textSurfaceObj1 = fontObj1.render('A', True, black, white)
    textRectObj1 = textSurfaceObj1.get_rect()
    textRectObj1.center = (coordinatePolygonA[0], coordinatePolygonA[1] - (font / 2))

    fontObj2 = pygame.font.SysFont("comicsansms", font)
    textSurfaceObj2 = fontObj2.render('E', True, black, white)
    textRectObj2 = textSurfaceObj2.get_rect()
    textRectObj2.center = (coordinatePolygonE[0] - (font / 2), coordinatePolygonE[1])

    fontObj3 = pygame.font.SysFont("comicsansms", font)
    textSurfaceObj3 = fontObj3.render('U', True, black, white)
    textRectObj3 = textSurfaceObj3.get_rect()
    textRectObj3.center = (coordinatePolygonU[0] + (font / 2), coordinatePolygonU[1])

    fontObj4 = pygame.font.SysFont("comicsansms", font)
    textSurfaceObj4 = fontObj4.render('I', True, black, white)
    textRectObj4 = textSurfaceObj4.get_rect()
    textRectObj4.center = (coordinatePolygonI[0], coordinatePolygonI[1] + (font / 2))

    fontObj5 = pygame.font.SysFont("comicsansms", font)
    textSurfaceObj5 = fontObj5.render('O', True, black, white)
    textRectObj5 = textSurfaceObj5.get_rect()
    textRectObj5.center = (coordinatePolygonO[0], coordinatePolygonO[1] + (font / 2))

def extractCatecianCoordinate(probabilityList):
    degreeOfA = [int(center_x * (1 - probabilityList[0]) + coordinatePolygonA[0] * probabilityList[0])
        , int(center_y * (1 - probabilityList[0]) + coordinatePolygonA[1] * probabilityList[0])]
    degreeOfE = [int(center_x * (1 - probabilityList[1]) + coordinatePolygonE[0] * probabilityList[1])
        , int(center_y * (1 - probabilityList[1]) + coordinatePolygonE[1] * probabilityList[1])]
    degreeOfI = [int(center_x * (1 - probabilityList[3]) + coordinatePolygonI[0] * probabilityList[3])
        , int(center_y * (1 - probabilityList[3]) + coordinatePolygonI[1] * probabilityList[3])]
    degreeOfO = [int(center_x * (1 - probabilityList[4]) + coordinatePolygonO[0] * probabilityList[4])
        , int(center_y * (1 - probabilityList[4]) + coordinatePolygonO[1] * probabilityList[4])]
    degreeOfU = [int(center_x * (1 - probabilityList[2]) + coordinatePolygonU[0] * probabilityList[2])
        , int(center_y * (1 - probabilityList[2]) + coordinatePolygonU[1] * probabilityList[2])]
    coordinatesOfDegree = [degreeOfA, degreeOfE, degreeOfI, degreeOfO, degreeOfU]
    return coordinatesOfDegree



pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:

def get_data(i):
    """
    parameter
    int : file number[31, 25]
    
    return
    numpy array with [1, 100]
    """
    
    data_file = Path("../RealTimeData/test"+str(i)+".csv")
    
    while(not data_file.exists()):
        pass
    
    # writing 지연시간 설정
    time.sleep(0.05) 
    
    data = np.genfromtxt("../RealTimeData/test"+str(i)+".csv", delimiter=',')
    
    data= np.vstack((data[4:6],data[7:9]))
    
    return np.reshape(data,[1,100])

In [5]:
if __name__ == "__main__":   
    
    # pygame initiation
    
    pygameInitialize()
    
    x= tf.placeholder(tf.float32, shape= [None, 20*20*5])
    y= tf.placeholder(tf.float32, shape= [None, 5])
    
    SAVER_DIR = "model/대현first"
    
    logit = model(x)
    
    saver = tf.train.Saver()
    checkpoint_path = os.path.join(SAVER_DIR, "model")
    ckpt = tf.train.get_checkpoint_state(SAVER_DIR)
    
    correct_percentage = tf.nn.softmax(logit)
    
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            print("Restored")
            fn=0
            data=np.array([])
            clock = pygame.time.Clock()
            
            while(True):
                start_time = time.time()#루프 한번 실행시간 측정
                
                new_data = get_data(fn)
                
                next_ = fn
                if data.size == 0:
                    data= new_data
                    continue
                else:
                    data= np.hstack((data, new_data))
                    print(data.size)
                    if data.size < 2000: # 2000개 (2초)의 데이터가 다 안 모아진 경우
                        continue 
                    elif data.size == 2000: # 2000개인 경우
                        pass
                    else: # 계속 들어와서 2100개가 된 경우
                        data = data[0][100:]
                        data=np.array([data]) 
                
                feed_input= data.tolist()
                print(fn)
       
                correctPercentage= sess.run(correct_percentage, feed_dict= {x: feed_input})
                print(correctPercentage)
                ProbabilityList= list(correctPercentage[0])
                fn+=1
                
                clock.tick(60)
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        sys.exit()

                key_event = pygame.key.get_pressed()

                if key_event[pygame.K_ESCAPE]:
                    sys.exit(0)

                screen.fill(white)
                pygame.draw.polygon(screen, black, (
                    coordinatePolygonA, coordinatePolygonE, coordinatePolygonI, coordinatePolygonO, coordinatePolygonU))

                for j in range(numberOfDegree):
                    for i in range(5):
                        # pygame.draw.circle(screen, colorList[numberOfDegree-j-1], (listsForDotsOfDegree[j][i][0], listsForDotsOfDegree[j][i][1]), 3)
                        pass
                    pygame.draw.line(screen, colorList[numberOfDegree - j - 1],
                                     listsForDotsOfDegree[j][0], listsForDotsOfDegree[j][1])
                    pygame.draw.line(screen, colorList[numberOfDegree - j - 1],
                                     listsForDotsOfDegree[j][0], listsForDotsOfDegree[j][2])
                    pygame.draw.line(screen, colorList[numberOfDegree - j - 1],
                                     listsForDotsOfDegree[j][2], listsForDotsOfDegree[j][3])
                    pygame.draw.line(screen, colorList[numberOfDegree - j - 1],
                                     listsForDotsOfDegree[j][3], listsForDotsOfDegree[j][4])
                    pygame.draw.line(screen, colorList[numberOfDegree - j - 1],
                                     listsForDotsOfDegree[j][1], listsForDotsOfDegree[j][4])
    
                for i in range(5):
                    print(ProbabilityList[i])
                    pygame.draw.circle(screen, red, extractCatecianCoordinate(ProbabilityList)[i], 7)
                # pygame.draw.circle(screen, white, (pos_x, pos_y), 7)
                screen.blit(textSurfaceObj1, textRectObj1)
                screen.blit(textSurfaceObj2, textRectObj2)
                screen.blit(textSurfaceObj3, textRectObj3)
                screen.blit(textSurfaceObj4, textRectObj4)
                screen.blit(textSurfaceObj5, textRectObj5)
                pygame.display.update()
                
                print("--- %s seconds ---" %(time.time() - start_time))

                

INFO:tensorflow:Restoring parameters from model\model-30
Restored
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
0


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node Conv2D (defined at <ipython-input-2-468bfc26c3a1>:59)  = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer, conv_1/filter/read)]]
	 [[{{node Softmax/_493}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_947_Softmax", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Conv2D', defined at:
  File "c:\users\bcilab\anaconda3\envs\uirp\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\bcilab\anaconda3\envs\uir